In [ ]:
import sys
import locale
import os
os.environ["RAY_TRAIN_V2_ENABLED"] = "0"

In [ ]:
sys.dont_write_bytecode = True
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install loguru==0.7.3 python-dotenv==1.0.1 PyYAML==6.0.2 torch==2.5.1 tqdm==4.67.1 typer==0.15.1 matplotlib==3.10.0 pyarrow==18.1.0 setuptools==75.1.0 protobuf==4.25.3 wandb==0.19.7 ultralytics==8.3.78 ray==2.43.0 albumentations==2.0.5

In [ ]:
#EyeConfig

import yaml
from pathlib import Path
from dotenv import dotenv_values
import torch

class EyeConfig:
    """Singleton class for managing project configuration and secrets."""
    _instance = None

    def __new__(cls):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        return cls._instance

    @staticmethod
    def load(config_file: str) -> dict:
        """Load and return configuration from YAML file."""
        with open(config_file, "r") as f:
            return yaml.safe_load(f)

    @staticmethod
    def get_device() -> str:
        try:
            return 0 if torch.cuda.is_available() else "cpu"
        except Exception as e:
            print(f"Error setting device: {e}")

    @staticmethod
    def get_wandb_key_colab() -> str:
        from google.colab import userdata # type: ignore
        if userdata.get("WANDB_API_KEY") is not None:
            return userdata.get("WANDB_API_KEY")
        else:
            raise ValueError("No WANDB key found")
    @staticmethod
    def get_wandb_key(path: Path = ".env") -> str:
        """Get W&B API key from Colab userdata or environment variable"""

        if not path.exists():
            raise FileNotFoundError(f"Could not find .env file at {path}")

        print(f"Loading secrets from {path}")

        secrets = dotenv_values(path)
        print(f"Found keys: {list(secrets.keys())}")

        if "WANDB_API_KEY" not in secrets:
            raise KeyError(f"WANDB_API_KEY not found in {path}. Available keys: {list(secrets.keys())}")

        return secrets['WANDB_API_KEY']

In [ ]:
#EyeBuilder
from pathlib import Path
from ultralytics import YOLO, settings
import wandb
from typing import Dict, Optional, Union
import logging
import os

class EyeBuilder:
    def __init__(self, model: YOLO, config: Dict, dataset_path: Optional[str] = None, wandb_key: Optional[str] = None, project_root: Optional[Union[str, Path]] = None) -> None:
        self.model = model
        self.config = config
        self.dataset_path = f"{config['dataset_name']}.yaml" if dataset_path is None else dataset_path
        self.wandb_key = wandb_key
        self.project_root = project_root

        self.logger = logging.getLogger(__name__)
        self.device = EyeConfig.get_device()

    def wandb_init(self, name) -> None:
        """Setup Weights & Biases tracking.

        Args:
            wandb_key: Optional API key for Weights & Biases
            project_root: Optional path to the project root directory. If not provided,
                        will use the current working directory.
        """
        # Use provided project root or fall back to current directory
        if self.project_root is None:
            self.project_root = Path(os.getcwd())

        # Create the full path for wandb directory
        wandb_dir = self.project_root / self.config["wandb"]["dir"]
        self.logger.info(f"Using wandb directory: {wandb_dir}")

        # Create directory if it doesn't exist
        wandb_dir.mkdir(parents=True, exist_ok=True)

        if self.wandb_key:
            wandb.login(key=self.wandb_key)

        wandb.init(
            project=self.config["project"],
            name=name,
            dir=str(wandb_dir),
        )

        settings.update({"wandb": True})

    def train(self, model):
        """Train the YOLO model with specified parameters."""
        self.wandb_init(f"{self.config['model_name']}_{self.config['dataset_name']}_train")

        train_kwargs = self.config["train"]
        results = model.train(
            data=str(self.dataset_path),
            device=self.device,
            **train_kwargs
        )
        wandb.finish()
        return results

    def tune(self, search_space):
        """Perform hyperparameter tuning on the model."""
        self.wandb_init(f"{self.config['model_name']}_{self.config['dataset_name']}_tune")

        tune = self.config["tune"]
        result_grid = self.model.tune(
            data=str(self.dataset_path),
            save_dir=Path(self.project_root / self.config["models_dir"]),
            device=self.device,
            project=self.config["project"],
            name=tune["name"],
            epochs=tune["epochs"],
            iterations=tune["iterations"],
            batch=tune["batch"],
            workers=tune["workers"],
            seed=tune["seed"],
            plots=tune["plots"],
            val=tune["val"],
            cos_lr=tune["cos_lr"],
            use_ray=tune["use_ray"],
            imgsz=tune["imgsz"],
            exist_ok=tune["exist_ok"],
            save=tune["save"],
            save_period=tune["save_period"],
            optimizer="AdamW",
            space=search_space,
        )
        wandb.finish()
        return result_grid

In [ ]:
# wandb_api_key = EyeConfig.get_wandb_key()
wandb_api_key = EyeConfig.get_wandb_key(Path("../.env"))

In [ ]:
from ray import tune

search_space = {
    "lr0": tune.choice([1e-4, 1e-3]),     # Keep it low for fine-tuning
    "lrf": tune.choice([0.01, 0.1]),          # Learning rate factor
    "momentum": tune.choice([0.8, 0.9, 0.95]),         # High momentum for stability
    "weight_decay": tune.choice([0.0, 0.001]),         # Minimal regularization
    "box": tune.uniform(2.0, 8.0),  # box loss gain
    "cls": tune.uniform(0.2, 2.0),  # cls loss gain (scale with pixels)
    "dfl": tune.uniform(3.0, 6.0),  # dfl loss gain
}

In [ ]:
config = EyeConfig.load("../config/config.yaml")
model_path = f"../{config['models_dir']}/{config['model_name']}.pt"
model = YOLO(model_path, task="detect")

In [ ]:
project_root = Path(os.getcwd()).parent
builder = EyeBuilder(model=model, config=config, wandb_key=wandb_api_key, project_root=project_root)

In [ ]:
result_grid = builder.tune(search_space)

In [ ]:
# from google.colab import userdata, files

# timestamp = time.strftime("%Y%m%d_%H%M%S")
# zip_filename = f"/content/EyeInTheSky_tune_{timestamp}.zip"

# !zip -r "$zip_filename" f/content/EyeInTheSky/tune

# files.download(zip_filename)

In [ ]:
# import shutil

# drive.mount('/content/drive')

# timestamp = time.strftime("%Y%m%d_%H%M%S")
# source_folder = '/content/EyeInTheSky/tune'
# destination_folder = f'/content/drive/My Drive/EyeInTheSky/tune_{timestamp}'

# shutil.copytree(source_folder, destination_folder)

In [ ]:
# plotter = EyePlotter()
# plotter.show_trial_results_metrics(results)
# plotter.show_results_plots(results, config["reports_dir"], config["name"])